# BackPropによるQAOA最適化

scikit-qulacsを用いてbackpropによる勾配計算が可能なQAOA回路を作成する。

## import

In [1]:
import numpy as np
import scipy
import matplotlib.pyplot as plt

from skqulacs.circuit import LearningCircuit

import sys
print(sys.path)

import made as MADE
import mcmc_function as mcmc
import ising_model as ising
from ising_model import Ising_model
import QAOA_function as qaoa
from QAOA_function import QAOA_ansatz

plt.rcParams["font.family"] = "Helvetica"
plt.rcParams['mathtext.fontset'] = 'cm'

seed = 729
rng = np.random.default_rng(seed)

['/Users/yuichiro_nakano/mydrive/Research/Fujii_Lab/Project/NN_proposal_MCMC_used_QAOA_sample/qaoa-made-mcmc/notebooks', '/Users/yuichiro_nakano/mydrive/Research/Fujii_Lab/Project/NN_proposal_MCMC_used_QAOA_sample/qaoa-made-mcmc/notebooks/src/lib', '/opt/anaconda3/envs/quantum/lib/python311.zip', '/opt/anaconda3/envs/quantum/lib/python3.11', '/opt/anaconda3/envs/quantum/lib/python3.11/lib-dynload', '', '/opt/anaconda3/envs/quantum/lib/python3.11/site-packages']


ModuleNotFoundError: No module named 'made'

## QAOA回路の定義

## 勾配計算テスト